# Introduction

Les équipes de Getaround souhaitent optimiser les tarifs en fonction des caractéristiques des véhicules. Pour cela, un dataset des prix et caractéristiques des véhicules a été fourni.

Sur cette base, il est demandé de réaliser un modèle de prédiction du tarif qu'il conviendra ensuite de mettre à disposition sous la forme d'une API.

In [1]:
# Import des librairies
import pandas as pd
import numpy as np
import seaborn as sn
import plotly.express as px

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import joblib

# Désactiver l'affichage des warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
prices = pd.read_csv('/content/src/get_around_pricing_project.csv')
prices.head()

,Unnamed: 0,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106
1,1,Citroën,13929,317,petrol,grey,convertible,True,True,False,False,False,True,True,264
2,2,Citroën,183297,120,diesel,white,convertible,False,False,False,False,True,False,True,101
3,3,Citroën,128035,135,diesel,red,convertible,True,True,False,False,True,True,True,158
4,4,Citroën,97097,160,diesel,silver,convertible,True,True,False,False,False,True,True,183


# Statistiques

In [3]:
print(f"Nombre de lignes : {prices.shape[0]}")
print()
print('---')
print()
print('Aperçu du dataset :')
print(prices.head())
print()
print('---')
print()
print('Statistiques basiques :')
print(prices.describe(include='all'))
print()
print('---')
print()
print('Pourcentage de valeurs manquantes :')
print((prices.isnull().sum()/len(prices)))


Nombre de lignes : 4843

---

Aperçu du dataset :
   Unnamed: 0 model_key  mileage  engine_power    fuel paint_color  \
0           0   Citroën   140411           100  diesel       black   
1           1   Citroën    13929           317  petrol        grey   
2           2   Citroën   183297           120  diesel       white   
3           3   Citroën   128035           135  diesel         red   
4           4   Citroën    97097           160  diesel      silver   

      car_type  private_parking_available  has_gps  has_air_conditioning  \
0  convertible                       True     True                 False   
1  convertible                       True     True                 False   
2  convertible                      False    False                 False   
3  convertible                       True     True                 False   
4  convertible                       True     True                 False   

   automatic_car  has_getaround_connect  has_speed_regulator  winter_tir

## Premiers enseignements

L'analyse globale du dataset nous permet déjà d'avoir un aperçu des features et du profil en moyenne des véhicules :
* Il n'y a pas de valeurs manquantes. La qualification des véhicules est optimale.
* La majorité des features est de type catégorielle, hormis les colonnes Unnamed: 0, mileage, engine_power et rental_price_per_day (la target du prix par jour).
* **Unnamed: 0** : Cette feature n'est pas utile et ne fait que dupliquer l'index des lignes. Il n'est pas utile de la conserver. On pourra la supprimer.
* **model_key** : C'est la marque de voiture. La plus nombreuse est Citroen.
* **mileage** : l'équivalent du kilométrage mais en miles. Le mileage moyen est de 140962 miles.
* **engine_power** : la moyenne est de l'équivalent de 128 chevaux
* **fuel** : Les véhicules sont majoritairement des diesel.
* **paint_color** : la couleur des voitures est majoritairement noire
* **car_type** : la plus grande catégorie de voitures est routière type break (estate)
* **private_parking_available** : La majorité des véhicules disposent d'un parking privé
* **has_gps** : la majorité des véhicules dispose du GPS
* **has_air_conditioning** : la majorité des véhicules n'est pas équipée de la climatisation
* **automatic_car** : la majorité des véhicules dispose d'une boite de vitesse manuelle
* **has_getaround_connect** : la majorité des véhicules ne dispose pas de l'option Connect
* **has_speed_regulator** : la majorité ne dispose pas du régulateur de vitesse
* **winter_tires** : la majorité dispose de pneus hiver
* **rental_price_per_day** : Il s'agit de la target à prédire. le prix moyen de location est de 121. Je penche plus pour 121$ car nous avons des références à l'unité de miles américains et qu'à l'époque Getaround n'était pas présente en France.

# Opérations de preprocessing

## Supprimer les colonnes inutiles

La seule feature inutile est Unnamed: 0. Je vais donc supprimer définitivement (inplace) cette colonne du dataframe.

In [4]:
# Cette feature est un doublon de l'index
prices['Unnamed: 0']

0          0
1          1
2          2
3          3
4          4
        ... 
4838    4838
4839    4839
4840    4840
4841    4841
4842    4842
Name: Unnamed: 0, Length: 4843, dtype: int64

In [5]:
prices.drop(prices.columns[0], axis=1, inplace=True)
prices.head()

,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106
1,Citroën,13929,317,petrol,grey,convertible,True,True,False,False,False,True,True,264
2,Citroën,183297,120,diesel,white,convertible,False,False,False,False,True,False,True,101
3,Citroën,128035,135,diesel,red,convertible,True,True,False,False,True,True,True,158
4,Citroën,97097,160,diesel,silver,convertible,True,True,False,False,False,True,True,183


## Création train/test set et pipeline

In [6]:
# Transforme colonnes boolean to string (en préparation du pipeline)
bool_cols = prices.select_dtypes(include=['bool']).columns.tolist()
for col in bool_cols:
    prices[col] = prices[col].astype('string')
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4843 entries, 0 to 4842
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   model_key                  4843 non-null   object
 1   mileage                    4843 non-null   int64 
 2   engine_power               4843 non-null   int64 
 3   fuel                       4843 non-null   object
 4   paint_color                4843 non-null   object
 5   car_type                   4843 non-null   object
 6   private_parking_available  4843 non-null   string
 7   has_gps                    4843 non-null   string
 8   has_air_conditioning       4843 non-null   string
 9   automatic_car              4843 non-null   string
 10  has_getaround_connect      4843 non-null   string
 11  has_speed_regulator        4843 non-null   string
 12  winter_tires               4843 non-null   string
 13  rental_price_per_day       4843 non-null   int64 
dtypes: int64

In [7]:
# Gestion de l'erreur ci-dessous dans le pipeline
# Found unknown categories ['Mazda', 'Mini'] in column 0 during transform
# On supprime du dataframe les model_key dans cette liste
marque_isolee = ['Mazda', 'Mini', 'Honda', 'Yamaha']

for marque in marque_isolee:
  prices = prices[prices['model_key'].str.contains(marque)==False ]

prices['model_key'].value_counts() # Doit renvoyer des valeurs ayant une occurence d'au moins 2

Citroën        969
Renault        916
BMW            827
Peugeot        642
Audi           526
Nissan         275
Mitsubishi     231
Mercedes        97
Volkswagen      65
Toyota          53
SEAT            46
Subaru          44
Opel            33
Ferrari         33
PGO             33
Maserati        18
Suzuki           8
Porsche          6
Ford             5
KIA Motors       3
Alfa Romeo       3
Fiat             2
Lamborghini      2
Lexus            2
Name: model_key, dtype: int64

In [8]:
# Separate target variable Y from features X
target_name = 'rental_price_per_day'

print("Separating labels from features...")
Y = prices.loc[:,target_name]
X = prices.drop(target_name, axis = 1) # All columns are kept, except the target
print("...Done.")
print(Y.head())
print()
print(X.head())
print()


Separating labels from features...
...Done.
0    106
1    264
2    101
3    158
4    183
Name: rental_price_per_day, dtype: int64

  model_key  mileage  engine_power    fuel paint_color     car_type  \
0   Citroën   140411           100  diesel       black  convertible   
1   Citroën    13929           317  petrol        grey  convertible   
2   Citroën   183297           120  diesel       white  convertible   
3   Citroën   128035           135  diesel         red  convertible   
4   Citroën    97097           160  diesel      silver  convertible   

  private_parking_available has_gps has_air_conditioning automatic_car  \
0                      True    True                False         False   
1                      True    True                False         False   
2                     False   False                False         False   
3                      True    True                False         False   
4                      True    True                False         False  

In [9]:
# Divide dataset Train set & Test set
print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0) # Ajout stratify car autrement problème au niveau des marques
print("...Done.")
print()

Dividing into train and test sets...
...Done.



In [10]:
# Identification des variables categorielles et numeriques
numeric_features = X.select_dtypes(include=['int64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object', 'string']).columns.tolist()
print(f"Numerical : {numeric_features}")
print(f"Categorielle : {categorical_features}")

Numerical : ['mileage', 'engine_power']
Categorielle : ['model_key', 'fuel', 'paint_color', 'car_type', 'private_parking_available', 'has_gps', 'has_air_conditioning', 'automatic_car', 'has_getaround_connect', 'has_speed_regulator', 'winter_tires']


In [11]:
# Création du pipeline pour les variables quantitatives
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler()) # pour normaliser les variables
])

# Création du pipeline pour les variables catégorielles
categorical_transformer = Pipeline(
    steps=[
    ('encoder', OneHotEncoder(drop='first')) # on encode les catégories sous forme de colonnes comportant des 0 et des 1
    ])

# On combine les pipelines dans un ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])



In [12]:
# Preprocessings on train set
print("Performing preprocessings on train set...")
print(X_train.head())
X_train = preprocessor.fit_transform(X_train)
print('...Done.')
print(X_train[0:5]) # MUST use this syntax because X_train is a numpy array and not a pandas DataFrame anymore
print()

# Preprocessings on test set
print("Performing preprocessings on test set...")
print(X_test.head())
X_test = preprocessor.transform(X_test) # Don't fit again !! The test set is used for validating decisions
# we made based on the training set, therefore we can only apply transformations that were parametered using the training set.
# Otherwise this creates what is called a leak from the test set which will introduce a bias in all your results.
print('...Done.')
print(X_test[0:5,:]) # MUST use this syntax because X_test is a numpy array and not a pandas DataFrame anymore
print()


Performing preprocessings on train set...
     model_key  mileage  engine_power    fuel paint_color   car_type  \
2038       BMW   102677           100  petrol       black  hatchback   
900    Peugeot   148986           100  diesel       black     estate   
933    Citroën   170500           135  diesel       black     estate   
2260       BMW   151334            85  diesel       white  hatchback   
3377   Citroën   207355           125  petrol       black      sedan   

     private_parking_available has_gps has_air_conditioning automatic_car  \
2038                     False   False                False         False   
900                       True    True                False         False   
933                       True    True                False         False   
2260                     False    True                False         False   
3377                     False   False                False         False   

     has_getaround_connect has_speed_regulator winter_tires  


# Baseline modèle

## Régression linéaire

Le principe est de lancer une régression linéaire simple en guise de baseline (record à battre par d'autres modèles).

In [13]:
# Train model
print("Train model...")
regressor = LinearRegression()
regressor.fit(X_train, Y_train)
print("...Done.")


Train model...
...Done.


In [14]:
# Predictions on training set
print("Predictions on training set...")
Y_train_pred = regressor.predict(X_train)
print("...Done.")
print(Y_train_pred)
print()


Predictions on training set...
...Done.
[ 89.91570775 101.9165638  105.47143502 ... 120.27291774 103.32796198
 117.77673395]



In [15]:
# Predictions on test set
print("Predictions on test set...")
Y_test_pred = regressor.predict(X_test)
print("...Done.")
print(Y_test_pred)
print()


Predictions on test set...
...Done.
[103.74178994 125.87361247 170.90842415 121.71869189 121.13797366
 153.94524436 126.66765177 117.08887832 118.16449858 130.59747082
 123.22035744  55.97455633 110.510192   102.32269967 114.44727575
 110.30068064 161.46015625 172.83272729 107.45831226 109.64272841
 113.13104299 136.10283625 157.10492577 120.07237119 103.92786397
 164.96924248 107.46094384 115.07837892 107.7441359  101.83290602
  98.48390708 120.12051293 111.14619851 205.59233011 113.1475759
  93.13819124 142.03470073 100.69161038 120.12369165  49.92227174
 159.88748557 127.46973911 110.06795099  64.55842592 110.59640663
 136.64824258 131.54537022 144.29108903 161.95166899 100.30271682
 101.1955531  169.89376365 139.46181424  96.63357622 121.75043347
  90.43290226 144.22048137 131.37834814 126.19089195 136.00701667
 161.84556716 145.35983124 108.63657653 149.59119607  99.64986188
 128.90100762 117.87985288 114.89273485  99.17162949  99.37319388
 134.55386029 102.13525927 107.94067511 1

In [16]:
# Print R^2 scores
print("R2 score on training set : ", r2_score(Y_train, Y_train_pred))
print("R2 score on test set : ", r2_score(Y_test, Y_test_pred))


R2 score on training set :  0.7161362935005403
R2 score on test set :  0.685345880372676


Notre modèle baseline fonctionne à un score R2 de 0.69. On peut imaginer une généralisation donnant des résultats pas trop éloignés.

## Observons l'importance des features

Pour cela, nous étudions les coefficients.

In [17]:
# List des coef
regressor.coef_

array([-13.47871766,  14.00616848,   9.81627553,   6.54152311,
         3.87252418,  16.78726602, -33.76881881, -14.93234157,
         8.00194587,   1.55485567,  17.74777604,  37.08559642,
        23.06986679,  18.7352934 ,  -2.26204883,  24.78302033,
        -6.53930821,   8.18359135,  20.12870528,  14.35594728,
        29.6557286 ,  16.70972473,  38.07348165,  36.78154111,
        23.82491002,  11.31008787,  39.90209741, -18.47419862,
        -1.69695358,  -4.23527129,  -0.63018886, -24.20873719,
        -3.25751125,  -5.10068181,   0.72163058,  -5.99472936,
         1.78177827,   6.30886811,  -8.81209513,  -7.30183458,
        -1.69434638,  -5.18614802,   4.2120322 , -32.35281747,
         1.30458269,  12.33115505,   0.84664234,   5.15048216,
         5.2193488 ,   4.84536906,  -4.32498991])

In [18]:
# Récupération des noms de mes colonnes
column_names = []
for name, pipeline, features_list in preprocessor.transformers_: # loop over pipelines
    if name == 'num': # if pipeline is for numeric variables
        features = features_list # just get the names of columns to which it has been applied
    else: # if pipeline is for categorical variables
        features = pipeline.named_steps['encoder'].get_feature_names_out() # get output columns names from OneHotEncoder
    column_names.extend(features) # concatenate features names

print("Names of columns corresponding to each coefficient: ", column_names)


Names of columns corresponding to each coefficient:  ['mileage', 'engine_power', 'model_key_Audi', 'model_key_BMW', 'model_key_Citroën', 'model_key_Ferrari', 'model_key_Fiat', 'model_key_Ford', 'model_key_KIA Motors', 'model_key_Lamborghini', 'model_key_Lexus', 'model_key_Maserati', 'model_key_Mercedes', 'model_key_Mitsubishi', 'model_key_Nissan', 'model_key_Opel', 'model_key_PGO', 'model_key_Peugeot', 'model_key_Porsche', 'model_key_Renault', 'model_key_SEAT', 'model_key_Subaru', 'model_key_Suzuki', 'model_key_Toyota', 'model_key_Volkswagen', 'fuel_electro', 'fuel_hybrid_petrol', 'fuel_petrol', 'paint_color_black', 'paint_color_blue', 'paint_color_brown', 'paint_color_green', 'paint_color_grey', 'paint_color_orange', 'paint_color_red', 'paint_color_silver', 'paint_color_white', 'car_type_coupe', 'car_type_estate', 'car_type_hatchback', 'car_type_sedan', 'car_type_subcompact', 'car_type_suv', 'car_type_van', 'private_parking_available_True', 'has_gps_True', 'has_air_conditioning_True',

In [19]:
# Create a pandas DataFrame
coefs = pd.DataFrame(index = column_names, data = regressor.coef_.transpose(), columns=["coefficients"])
coefs.sort_values('coefficients', ascending=False)

,coefficients
fuel_hybrid_petrol,39.902097
model_key_Suzuki,38.073482
model_key_Maserati,37.085596
model_key_Toyota,36.781541
model_key_SEAT,29.655729
model_key_Opel,24.783020
model_key_Volkswagen,23.824910
model_key_Mercedes,23.069867
model_key_Porsche,20.128705
model_key_Mitsubishi,18.735293


In [20]:
# Compute abs() and sort values
feature_importance = abs(coefs)
feature_importance = feature_importance.sort_values(by = 'coefficients', ascending=False)

In [21]:
# Plot coefficients
fig = px.bar(feature_importance, orientation = 'h')
fig.update_layout(showlegend = False,
                  margin = {'l': 120} # to avoid cropping of column names
                 )
fig.update_yaxes(autorange="reversed")
fig.show()


En conclusion, les 10 variables ayant le plus d'impact sur le prix sont les suivantes :
1. Véhicules hybrides
2. 3 marques : Maserati, Fiat, Seat
3. La couleur verte
4. 5 marques : Mercedes, Mitsubishi, Lexus, Subaru et Renault

Le kilométrage (mileage) n'arrive qu'en 10e position

# Modèle avec régularisations

Il n'y a pas besoin d'appliquer à nouveau les preprocessing car c'est déjà fait.

## Ridge

In [22]:
# Perform grid search
print("Grid search...")
regressor = Ridge()
# Grid of values to be tested
params = {
    'alpha': [2.03, 2.05, 2.07] # 0 corresponds to no regularization
}
gridsearch_ridge = GridSearchCV(regressor, param_grid = params, cv = 10) # cv : the number of folds to be used for CV
gridsearch_ridge.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch_ridge.best_params_)
print("Best R2 score : ", gridsearch_ridge.best_score_)


Grid search...
...Done.
Best hyperparameters :  {'alpha': 2.05}
Best R2 score :  0.7097732362199933


Le résultat de Ridge nous donne un R2 de 0.70 meilleur que la baseline et attesté par la gridsearch.

## Lasso

In [23]:
# Perform grid search
print("Grid search...")
regressor = Lasso()
# Grid of values to be tested
params = {
    'alpha': [51, 52, 53] # 0 corresponds to no regularization
}
gridsearch_lasso = GridSearchCV(regressor, param_grid = params, cv = 30) # cv : the number of folds to be used for CV
gridsearch_lasso.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch_lasso.best_params_)
print("Best R2 score : ", gridsearch_lasso.best_score_)

Grid search...
...Done.
Best hyperparameters :  {'alpha': 51}
Best R2 score :  -0.0075573629376059605


La régularisation Lasso ne donne pas de bons scores. En fait, nous n'avons certainement pas suffisamment de lignes pour laisser le soin à Lasso d'aller discriminer (par un coefficient nul) les colonnes inutiles.

## Export du modèle gagnant Ridge

In [24]:
# Export du preprocessing
joblib.dump(preprocessor,'preprocessor.joblib')

['preprocessor.joblib']

In [25]:
# Export du modèle gagnant au format en 1 seul fichier
joblib.dump(gridsearch_ridge, 'getaround_price_model.joblib')

['getaround_price_model.joblib']